In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import exovar
import LevelsCreater as lc
from functions import series_to_supervised

levels = lc.LevelsCreater()
sale=pd.read_csv(os.path.join('Data2','sales_train_evaluation.csv'))
s=levels.get_level(sale,1)
calendar=pd.read_csv(os.path.join('Data2','calendar.csv'))
s=s.T
s=s.rename(columns={s.columns[0]:'zero'})
s=s.T
ex=exovar.exovar()
salecal=ex.salecal(s,calendar,node=0)
x,y,e=series_to_supervised(salecal,5,1,parse=True,dropnan=True)
x

,zero(t-5),zero(t-3),zero(t-1),zero(t-4),zero(t-2)
5,32631.0,23783.0,19146.0,31749.0,25412.0
6,31749.0,25412.0,29211.0,23783.0,19146.0
7,23783.0,19146.0,28010.0,25412.0,29211.0
8,25412.0,29211.0,37932.0,19146.0,28010.0
9,19146.0,28010.0,32736.0,29211.0,37932.0
...,...,...,...,...,...
1936,44073.0,59921.0,38777.0,54308.0,42362.0
1937,54308.0,42362.0,37096.0,59921.0,38777.0
1938,59921.0,38777.0,36963.0,42362.0,37096.0
1939,42362.0,37096.0,42552.0,38777.0,36963.0


In [38]:
from sklearn.gaussian_process.kernels import WhiteKernel, ExpSineSquared, ConstantKernel,RBF
from sklearn.gaussian_process import GaussianProcessRegressor

k0 = WhiteKernel(noise_level=0.3**2, noise_level_bounds=(0.1**2, 0.5**2))

k1 = ConstantKernel(constant_value=2) * \
    ExpSineSquared(length_scale=1.0, periodicity=40, periodicity_bounds=(35, 45))

k2 = ConstantKernel(constant_value=10, constant_value_bounds=(1e-2, 1e3)) * RBF(length_scale=100.0, length_scale_bounds=(1, 1e4)) 

k3 = ConstantKernel(constant_value=1) * ExpSineSquared(length_scale=1.0, periodicity=12, periodicity_bounds=(10, 15))

kernel_3  = k0 + k1 + k2 + k3

gp3 = GaussianProcessRegressor(
    kernel=kernel_3, 
    n_restarts_optimizer=10, 
    normalize_y=True,
    alpha=0.0
)



In [39]:
X=x.join(e)
Y=y
X_train=X.iloc[:-28]
Y_train=Y.iloc[:-28]
X_test=X.iloc[-28:]
Y_test=Y.iloc[-28:]

gp3.fit(X_train, Y_train)

GaussianProcessRegressor(alpha=0.0, copy_X_train=True,
                         kernel=WhiteKernel(noise_level=0.09) + 1.41**2 * ExpSineSquared(length_scale=1, periodicity=40) + 3.16**2 * RBF(length_scale=100) + 1**2 * ExpSineSquared(length_scale=1, periodicity=12),
                         n_restarts_optimizer=10, normalize_y=True,
                         optimizer='fmin_l_bfgs_b', random_state=None)

In [40]:
y_pred, y_std = gp3.predict(X_test, return_std=True)
print(len(y_pred))
print(len(Y_test))

28
28


In [41]:
from functions import RMSSE
s = s.transpose()
df = pd.DataFrame(s[s.columns[0]])
rmsse=RMSSE(y_pred,Y_test,df.values)
print(rmsse)

1.979914622618084


In [35]:
y_pred

array([34371.73304729, 34371.73437117, 34371.73479202, 34371.73448772,
       34371.73439682, 34371.73615908, 34371.73510911, 34371.7339124 ,
       34371.7345839 , 34371.73415042, 34371.73595209, 34371.73417534,
       34371.73346238, 34371.73443861, 34371.73465625, 34371.73558078,
       34371.73590165, 34371.73534148, 34371.73501331, 34371.734853  ,
       34371.73476344, 34371.73443768, 34371.73546284, 34371.7346677 ,
       34371.73560855, 34371.73564044, 34371.73447764, 34371.7347582 ])

In [36]:
Y_test

1913    38793
1914    35487
1915    34445
1916    34732
1917    42896
1918    50429
1919    53032
1920    43181
1921    44314
1922    39601
1923    40763
1924    43805
1925    54239
1926    45609
1927    46400
1928    39379
1929    42248
1930    40503
1931    44073
1932    54308
1933    59921
1934    42362
1935    38777
1936    37096
1937    36963
1938    42552
1939    51518
1940    54338
Name: zero(t), dtype: int64